## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup

import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,city_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,21.00,93,90,9.22,light snow
1,1,Talcahuano,CL,-36.7167,-73.1167,60.80,82,90,6.91,overcast clouds
2,2,Hermanus,ZA,-34.4187,19.2345,68.16,80,3,3.53,clear sky
3,3,Mairana,BO,-18.1167,-63.9333,60.06,72,88,2.62,overcast clouds
4,4,Lagoa,PT,39.0500,-27.9833,58.55,85,91,18.75,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefer_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

prefer_cities_df

,city_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
18,18,Cockburn Town,TC,21.4612,-71.1419,76.93,85,15,18.41,few clouds
20,20,Valenca,BR,-13.3703,-39.0731,75.99,88,98,3.40,overcast clouds
30,30,Carnarvon,AU,-24.8667,113.6333,84.20,100,75,21.85,broken clouds
31,31,Hithadhoo,MV,-0.6000,73.0833,82.69,76,90,16.31,overcast clouds
33,33,Bermejo,AR,-32.8833,-68.8000,75.20,43,0,5.75,clear sky
...,...,...,...,...,...,...,...,...,...,...
674,674,Utete,TZ,-7.9860,38.7580,81.68,77,29,3.18,scattered clouds
675,675,Matola,MZ,-25.9622,32.4589,81.00,89,75,6.91,broken clouds
677,677,Natal,BR,-5.7950,-35.2094,77.00,83,20,2.30,few clouds
683,683,Boa Vista,BR,2.8197,-60.6733,77.00,78,0,8.05,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
prefer_cities_df.isnull().sum()

city_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefer_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ''
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Cockburn Town,TC,76.93,few clouds,21.4612,-71.1419,
20,Valenca,BR,75.99,overcast clouds,-13.3703,-39.0731,
30,Carnarvon,AU,84.20,broken clouds,-24.8667,113.6333,
31,Hithadhoo,MV,82.69,overcast clouds,-0.6000,73.0833,
33,Bermejo,AR,75.20,clear sky,-32.8833,-68.8000,
...,...,...,...,...,...,...,...
674,Utete,TZ,81.68,scattered clouds,-7.9860,38.7580,
675,Matola,MZ,81.00,broken clouds,-25.9622,32.4589,
677,Natal,BR,77.00,few clouds,-5.7950,-35.2094,
683,Boa Vista,BR,77.00,clear sky,2.8197,-60.6733,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel in {row['City']}, {row['Country']} not found... skipping.")
        

Hotel in Beloha, MG not found... skipping.
Hotel in Ikongo, MG not found... skipping.
Hotel in Luba, GQ not found... skipping.
Hotel in Lagos, NG not found... skipping.
Hotel in Gamba, GA not found... skipping.
Hotel in Thinadhoo, MV not found... skipping.
Hotel in Barra Patuca, HN not found... skipping.
Hotel in Kimamba, TZ not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace('',value = pd.np.nan)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

C:\Users\Hayden\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Cockburn Town,TC,76.93,few clouds,21.4612,-71.1419,Osprey Beach Hotel
20,Valenca,BR,75.99,overcast clouds,-13.3703,-39.0731,Hotel Galeão
30,Carnarvon,AU,84.20,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
31,Hithadhoo,MV,82.69,overcast clouds,-0.6000,73.0833,Scoop Guest House
33,Bermejo,AR,75.20,clear sky,-32.8833,-68.8000,Hotel Mendoza
...,...,...,...,...,...,...,...
674,Utete,TZ,81.68,scattered clouds,-7.9860,38.7580,River View Lodge
675,Matola,MZ,81.00,broken clouds,-25.9622,32.4589,Guesthouse Yuba
677,Natal,BR,77.00,few clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
683,Boa Vista,BR,77.00,clear sky,2.8197,-60.6733,Aipana Plaza Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# call figure
fig

Figure(layout=FigureLayout(height='420px'))